In [1]:
################ Letter-Recognition #################
# Disable info messages from the tesnorflow
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
################## Import the libraries #####################
import numpy as np
import pandas as pd
import keras
from keras.models import Model, Sequential
from keras.utils import to_categorical
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D,MaxPooling2D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler,MinMaxScaler,scale
from sklearn.metrics import accuracy_score
from scipy.ndimage import zoom

In [2]:
############### Load the dataset ###################
path = 'HIGGS.csv'
classes = 2
data=pd.read_csv(path, header=None)
samples = data.sample(n=700000)
del data
print(samples)

           0         1         2         3         4         5         6   \
3667426   0.0  1.469563  0.262917  0.991399  1.400955  1.570351  0.704459   
3238447   0.0  1.066943  1.709261  0.086975  0.522069  1.717988  0.504756   
370091    1.0  1.390869 -0.407173  0.273408  0.439804 -1.413787  1.313922   
10685439  0.0  1.496465  1.147281  1.041336  0.595993  0.006550  0.528574   
677259    1.0  1.698324 -1.038305 -1.598799  0.758006 -0.951673  1.103409   
...       ...       ...       ...       ...       ...       ...       ...   
9678150   1.0  0.535302  0.941774 -1.431786  0.205735  0.067287  0.966548   
4255222   0.0  0.442516 -0.504570 -0.072928  1.070002 -1.524339  1.021604   
4460195   0.0  0.588740 -0.066284  1.651684  1.449585 -1.021703  0.611295   
5105307   1.0  0.600453  0.636921 -1.187647  0.767268  1.026184  0.845351   
9598193   0.0  0.761501 -1.945070 -1.037834  1.491004 -0.230808  0.614776   

                7         8         9   ...        19        20        21  

In [3]:
X =samples.iloc[:,1:]
Y =samples.iloc[:,0]
X = MinMaxScaler().fit_transform(X)
classes = 2
Y = to_categorical(Y,classes)

In [4]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(560000, 28) (560000, 2)
(140000, 28) (140000, 2)


In [5]:
y_train = np.argmax(Y_train,axis=-1)
from tab2img.converter import Tab2Img
model = Tab2Img()
X_train = model.fit_transform(X_train, y_train)
X_test = model.transform(X_test)

In [6]:
from PIL import Image
x_train = []
x_test = []
for i in range(len(X_train)):
    img = Image.fromarray(X_train[i])
    imgs = img.resize(size=(32, 32))
    imgs = np.array(imgs)
    x_train.append(np.repeat(imgs[:, :, np.newaxis], 3, axis=2))
x_train = np.array(x_train,dtype='float32')

for i in range(len(X_test)):
    img = Image.fromarray(X_test[i])
    imgs = img.resize(size=(32, 32))
    imgs = np.array(imgs)
    x_test.append(np.repeat(imgs[:, :, np.newaxis], 3, axis=2))
x_test = np.array(x_test,dtype='float32')

print(x_train.shape,x_test.shape)

(560000, 32, 32, 3) (140000, 32, 32, 3)


In [7]:
############### Feature-extraction ###############
# pretrained model for FE
pretrained_model = keras.applications.DenseNet121(input_shape=(32,32,3), weights=None, include_top=False)
x = Flatten()(pretrained_model.output)
x = Dense(512, activation='relu',name='flattenn')(x)
x = Dropout(0.5)(x)
prediction = Dense(classes, activation='softmax')(x)
model = Model(inputs=pretrained_model.input, outputs=prediction)
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)
# fit training data and validate on the testing data
model.fit(x_train,Y_train,batch_size=256, epochs=20,verbose=1,validation_data=(x_test,Y_test))

Epoch 1/20
2188/2188 [==============================] - 162s 74ms/step - loss: 0.6151 - accuracy: 0.6589 - val_loss: 0.5980 - val_accuracy: 0.6720
Epoch 2/20
2188/2188 [==============================] - 160s 73ms/step - loss: 0.5620 - accuracy: 0.7116 - val_loss: 0.6076 - val_accuracy: 0.6570
Epoch 3/20
2188/2188 [==============================] - 161s 74ms/step - loss: 0.5513 - accuracy: 0.7184 - val_loss: 0.7195 - val_accuracy: 0.6496
Epoch 4/20
2188/2188 [==============================] - 161s 74ms/step - loss: 0.5443 - accuracy: 0.7232 - val_loss: 0.5549 - val_accuracy: 0.7114
Epoch 5/20
2188/2188 [==============================] - 161s 74ms/step - loss: 0.5363 - accuracy: 0.7291 - val_loss: 0.5631 - val_accuracy: 0.7076
Epoch 6/20
2188/2188 [==============================] - 160s 73ms/step - loss: 0.5266 - accuracy: 0.7351 - val_loss: 0.5344 - val_accuracy: 0.7264
Epoch 7/20
2188/2188 [==============================] - 161s 73ms/step - loss: 0.5186 - accuracy: 0.7402 - val_loss: 0

In [8]:
score = model.evaluate(x_test,Y_test)
acc = model.evaluate(x_train,Y_train)
print('training accuracy: ',acc[1])
print('testing accuracy: ',score[1])

17500/17500 [==============================] - 149s 9ms/step - loss: 0.3688 - accuracy: 0.8280
training accuracy:  0.8279678821563721
testing accuracy:  0.7363214492797852
